# 获取所有相关用户个人信息

已有用户 id，获取用户粉丝数，关注数等信息。up 主还需单独获取其性别，生日等信息，评论者这部分信息在拿评论信息时已得到，直接合并即可，对应结果保存在 `../data/user/user_full.csv`（其中 `is_up` 字段用来区分 up 主还是普通评论者）

In [51]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time 
import numpy as np
import os
import datetime
import random

In [52]:
ua_list = ["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
           "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
           "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
           "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
           "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
           "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
           "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
           "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
           "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
           "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
           "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
           "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
           "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
           "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
           "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
           "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
           "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
           "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
           ]
headers1 = {
    'user-agent': random.choice(ua_list)
}

In [57]:
def get_user(uid, type, low = 0.1, high = 0.4):
    time.sleep(np.random.uniform(low=low, high=high, size=1).item())
    if type == "info" or type == "i":
        # 频率快容易别拦截，只对up主获取该链接内容
        url = "https://api.bilibili.com/x/space/acc/info?mid={}&jsonp=jsonp".format(uid)
    elif type == "follow" or type == "f":
        # 一般不会有问题
        url = "https://api.bilibili.com/x/relation/stat?vmid={}&jsonp=jsonp".format(uid)
    elif type == "number" or type == "n":
        # 极容易被拦截，后放弃
        url = "https://api.bilibili.com/x/space/upstat?mid={}&jsonp=jsonp".format(uid)
    else:
        print("Wrong type")
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers1, data=payload)
    try:
        info_dic = json.loads(response.text)['data']
    except:
        print(response.text)
        print(uid, "fail at:\t", url)
        return {"mid": uid}
    return info_dic

# Upper

粉丝数和关注数一般不会出问题，但是其他信息容易被拦截，最好间隔时间设置大一点，一次运行完，不要连续 debug

In [ ]:
df = pd.read_csv("../data/video/video_final.csv", encoding="UTF-8")
uid_list = set(df["u_id"])

In [ ]:
len(uid_list)

567

In [ ]:
uinfo = {"follow":[], "info": []}
for uid in tqdm(uid_list):
    for key in uinfo.keys():
        uinfo[key].append(get_user(uid, key))
    if not uinfo['info'][-1]:
        print(uid)
        print(len(uinfo['info']))
        break

 46%|████▌     | 262/567 [05:34<05:54,  1.16s/it]

{"code":-404,"message":"啥都木有","ttl":1}
451972012 fail at:	 https://api.bilibili.com/x/space/acc/info?mid=451972012&jsonp=jsonp


100%|██████████| 567/567 [11:39<00:00,  1.23s/it]


In [ ]:
udf = pd.merge(pd.DataFrame(uinfo['follow']), pd.DataFrame(uinfo['info']))
udf.to_csv("../data/user/user_up.csv", index = False, header = True, encoding = "utf-8")

# Comment User

此处仅获取粉丝数和关注数

In [53]:
path_r = "../data/reply"
replies = os.listdir(path_r)
uid_list = []
for file in tqdm(replies):
    try:
        try:
            r = pd.read_csv(path_r + "/" + file)
        except:
            r = pd.read_csv(path_r + "/" + file, encoding="ANSI")
    except:
        print(file)
    uid_list.extend(r['mid'])

100%|██████████| 961/961 [00:22<00:00, 42.57it/s]


In [54]:
len(uid_list), len(set(uid_list))

(908079, 254634)

In [55]:
uid_list = list(set(uid_list))

In [60]:
# 每 1000 个会保存一个临时文件保存中间结果，记得删除
uinfo = {"follow":[], "info": []}
i = 0
f = open("../logs/user_log.txt", "a")
f.write("\n===================================================\n")
f.close()
for uid in tqdm(uid_list):
    uinfo['follow'].append(get_user(uid, "follow", 0.05, 0.25))
    if not uinfo['follow'][-1]:
        print("follow ", uid, i)
        f = open("../logs/user_log.txt", "a")
        f.write("\n{}\t exmpty follow of: {} (No.{})".format(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), uid, i))
        f.close()
        uinfo['follow'][-1] = {"mid":uid}
        time.sleep(5)
    # 临时保存
    if len(uinfo['follow']) % 1000 == 0:
        udf = pd.DataFrame(uinfo['follow'])
        udf.to_csv("../data/user/user_com_{}_{}.csv".format(len(uinfo['follow']), datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')),header=True, index=False, encoding="utf-8")
udf = pd.DataFrame(uinfo['follow'])
udf.to_csv("../data/user/user_com.csv".format(file[6:-4]),header=True, index=False, encoding="utf-8")

 12%|█▏        | 7915/66000 [36:14<4:09:53,  3.87it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 22%|██▏       | 14620/66000 [1:06:51<2:53:02,  4.95it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 55%|█████▍    | 36246/66000 [2:45:54<2:29:46,  3.31it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--N